In [2]:
from pathlib import Path
import duckdb
import pandas as pd

REPO_ROOT = Path(r"C:\Dev\youtube-trending-app")  # adjust if needed
DB_PATH = REPO_ROOT / "data" / "processed" / "trending.duckdb"

con = duckdb.connect(str(DB_PATH), read_only=True)

def q(sql, params=None):
    return con.execute(sql, params or []).df()

q("SELECT 1 AS ok")


,ok
0,1


In [13]:
import pandas as pd

pd.set_option("display.max_columns", None)      # show all columns
pd.set_option("display.width", 200)             # wider output
pd.set_option("display.max_colwidth", 80)       # limit long text so it doesn't explode
pd.set_option("display.expand_frame_repr", False)


In [5]:
q("""
SELECT
  count(*) AS rows,
  count(DISTINCT video_id) AS distinct_videos,
  count(DISTINCT channel_id) AS distinct_channels,
  count(DISTINCT video_trending_country) AS countries,
  min(video_trending_date) AS min_date,
  max(video_trending_date) AS max_date
FROM trending
WHERE video_trending_date IS NOT NULL;
""")


,rows,distinct_videos,distinct_channels,countries,min_date,max_date
0,7614272,692565,100419,110,2024-10-12,2026-01-31


Top 20 countries by volume

In [6]:
q("""
SELECT video_trending_country, count(*) AS rows
FROM trending
WHERE video_trending_date IS NOT NULL
GROUP BY 1
ORDER BY rows DESC
LIMIT 20;
""")


,video_trending_country,rows
0,Austria,94796
1,Japan,94794
2,South Korea,94793
3,Mexico,94792
4,Australia,94792
5,Canada,94790
6,Germany,94790
7,Brazil,94790
8,United Kingdom,94787
9,United States,94787


Date range per country (uses your view)

In [7]:
q("""
SELECT *
FROM v_available_dates
ORDER BY rows DESC
LIMIT 20;
""")


,video_trending_country,min_date,max_date,rows
0,Austria,2024-10-12,2026-01-31,94796
1,Japan,2024-10-12,2026-01-31,94794
2,South Korea,2024-10-12,2026-01-31,94793
3,Mexico,2024-10-12,2026-01-31,94792
4,Australia,2024-10-12,2026-01-31,94792
5,Brazil,2024-10-12,2026-01-31,94790
6,Canada,2024-10-12,2026-01-31,94790
7,Germany,2024-10-12,2026-01-31,94790
8,United Kingdom,2024-10-12,2026-01-31,94787
9,United States,2024-10-12,2026-01-31,94787


Cell 5 — Missingness snapshot (important fields)

In [8]:
q("""
SELECT
  round(100.0 * avg(CAST(video_tags IS NULL AS INT)), 2) AS pct_video_tags_null,
  round(100.0 * avg(CAST(video_description IS NULL AS INT)), 2) AS pct_video_desc_null,
  round(100.0 * avg(CAST(channel_country IS NULL AS INT)), 2) AS pct_channel_country_null,
  round(100.0 * avg(CAST(video_like_count IS NULL AS INT)), 2) AS pct_like_null,
  round(100.0 * avg(CAST(video_comment_count IS NULL AS INT)), 2) AS pct_comment_null
FROM trending;
""")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,pct_video_tags_null,pct_video_desc_null,pct_channel_country_null,pct_like_null,pct_comment_null
0,27.7,9.79,17.8,0.95,0.58


In [9]:
q("""
SELECT
  count(*) AS rows,
  sum(CAST(video_trending_date IS NULL AS INT)) AS null_trending_date_rows
FROM trending;
""")


,rows,null_trending_date_rows
0,7614272,0.0


Find countries that have any NULL dates (data corruption check)

In [3]:
q("""SELECT video_trending_country, count(*) AS rows
FROM trending
WHERE video_trending_date IS NULL
GROUP BY 1
ORDER BY rows DESC;
""")

,video_trending_country,rows


How many columns? What are the column names?

In [10]:
q("PRAGMA table_info('trending');")


,cid,name,type,notnull,dflt_value,pk
0,0,video_id,VARCHAR,False,None,False
1,1,video_published_at,TIMESTAMP WITH TIME ZONE,False,None,False
2,2,video_trending_date,DATE,False,None,False
3,3,video_trending_country,VARCHAR,False,None,False
4,4,channel_id,VARCHAR,False,None,False
5,5,video_title,VARCHAR,False,None,False
6,6,video_description,VARCHAR,False,None,False
7,7,video_default_thumbnail,VARCHAR,False,None,False
8,8,video_category_id,INTEGER,False,None,False
9,9,video_tags,VARCHAR,False,None,False


In [14]:
q("""
SELECT *
FROM trending
LIMIT 5;
""")


,video_id,video_published_at,video_trending_date,video_trending_country,channel_id,video_title,video_description,video_default_thumbnail,video_category_id,video_tags,video_duration,video_dimension,video_definition,video_licensed_content,video_view_count,video_like_count,video_comment_count,channel_title,channel_description,channel_custom_url,channel_published_at,channel_country,channel_view_count,channel_subscriber_count,channel_have_hidden_subscribers,channel_video_count,channel_localized_title,channel_localized_description
0,bB3-CUMERIU,2024-10-10 20:00:06-04:00,2024-10-12,United Arab Emirates,UCNYi_zGmR519r5gYdOKLTjQ,JENNIE - Mantra (Official Music Video),Official music video for “Mantra” by JENNIE out now: https://jennie.lnk.to/m...,https://i.ytimg.com/vi/bB3-CUMERIU/default.jpg,<NA>,"JENNIE,Mantra,제니,Jennie Mantra,Jennie from Blackpink,Blackpink,dance,dancer,...",PT2M28S,2d,hd,False,20535235,2042255,152933,JENNIE,NaN,@jennierubyjane,2021-01-13 01:19:55.866890-05:00,NaN,464615150,11600000,False,43,JENNIE,NaN
1,5ObJt_71AYc,2024-10-10 22:59:21-04:00,2024-10-12,United Arab Emirates,UCzU8-lZlRfkV3nj0RzAZdrQ,CHILE vs. BRASIL [1-2] | RESUMEN | ELIMINATORIAS SUDAMERICANAS | FECHA 9,#Chile #Brasil #eliminatoriassudamericanas #CreeEnGrande #CONMEBOL \n\n⭐🇧🇷 V...,https://i.ytimg.com/vi/5ObJt_71AYc/default.jpg,<NA>,"fútbol,pelota,gol,copa,libertadores,sudamericana,eliminatorias,mundial,conme...",PT10M8S,2d,hd,True,3966042,<NA>,2549,CONMEBOL,Canal Oficial de la Confederación Sudamericana de Fútbol. #CreeEnGrande | Ca...,@conmebol,2014-02-19 15:24:31-05:00,NaN,399046746,1610000,False,4637,CONMEBOL,Canal Oficial de la Confederación Sudamericana de Fútbol. #CreeEnGrande | Ca...
2,zfb0whgBBA8,2024-10-11 07:07:25-04:00,2024-10-12,United Arab Emirates,UCgGYPnVJytkr6sVNLQ-l0zQ,تحدي الحقيبة مع ابو عبير 🔥 | سلسلتنا #2,"1,375,547 مشترك❤️\n\nـــــــــــــــــــــــــــــــــــــــــــــــــــــــ...",https://i.ytimg.com/vi/zfb0whgBBA8/default.jpg,<NA>,"تشكيله,فيفا,فيفا20,التمت,درافت,زلاتان,ام,جي,ملتي,جيمرز,لاتيني,ترامب,كورة,قدم...",PT43M24S,2d,hd,True,853167,101155,10541,LLE عبدالاله,صانع محتوى وانزل مقاطع بشكل غير منظم 🌚,@mrlle99,2012-08-01 12:24:26-04:00,Saudi Arabia,114331110,1380000,False,314,LLE عبدالاله,صانع محتوى وانزل مقاطع بشكل غير منظم 🌚
3,SJfoPdeOPCQ,2024-10-10 20:10:10-04:00,2024-10-12,United Arab Emirates,UCzU8-lZlRfkV3nj0RzAZdrQ,VENEZUELA vs. ARGENTINA [1-1] | RESUMEN | ELIMINATORIAS SUDAMERICANAS | FECHA 9,#Venezuela #Argentina #eliminatoriassudamericanas #CreeEnGrande #CONMEBOL \n...,https://i.ytimg.com/vi/SJfoPdeOPCQ/default.jpg,<NA>,"fútbol,pelota,gol,copa,libertadores,sudamericana,eliminatorias,mundial,conme...",PT10M9S,2d,hd,True,3758707,<NA>,3115,CONMEBOL,Canal Oficial de la Confederación Sudamericana de Fútbol. #CreeEnGrande | Ca...,@conmebol,2014-02-19 15:24:31-05:00,NaN,399046746,1610000,False,4637,CONMEBOL,Canal Oficial de la Confederación Sudamericana de Fútbol. #CreeEnGrande | Ca...
4,UVb6QOKy0bI,2024-10-09 08:30:27-04:00,2024-10-12,United Arab Emirates,UCOzubmwpVZI7gD0Jf7Bk3Aw,Bougainvillea Trailer | October 17 | Jyothirmayi | Kunchacko Boban | Amal Ne...,Presenting The Official Trailer of Bougainvillea Directed by Amal Neerad\n\n...,https://i.ytimg.com/vi/UVb6QOKy0bI/default.jpg,<NA>,NaN,PT2M12S,2d,hd,True,1730189,67522,2869,Amal Neerad Productions,Official YouTube Channel Of Amal Neerad Productions .,@amalneeradproductions5264,2017-05-31 10:31:01-04:00,NaN,19991522,40600,False,56,Amal Neerad Productions,Official YouTube Channel Of Amal Neerad Productions .


Daily volume sanity (should be ~200/day/country)

In [15]:
q("""
SELECT video_trending_country, video_trending_date, count(*) AS rows_that_day
FROM trending
GROUP BY 1,2
ORDER BY rows_that_day DESC
LIMIT 20;
""")


,video_trending_country,video_trending_date,rows_that_day
0,Brazil,2025-02-06,200
1,Colombia,2025-02-10,200
2,Algeria,2025-05-02,200
3,Colombia,2025-02-08,200
4,Australia,2025-04-28,200
5,Morocco,2025-02-07,200
6,Austria,2025-05-02,200
7,Colombia,2025-02-11,200
8,Bolivia,2025-05-01,200
9,Colombia,2025-02-12,200


Days trended” distribution (stickiness)

In [16]:
q("""
WITH per_video_country AS (
  SELECT video_id, video_trending_country, count(DISTINCT video_trending_date) AS days_trended
  FROM trending
  GROUP BY 1,2
)
SELECT
  approx_quantile(days_trended, 0.50) AS p50,
  approx_quantile(days_trended, 0.90) AS p90,
  approx_quantile(days_trended, 0.99) AS p99,
  max(days_trended) AS max_days
FROM per_video_country;
""")


,p50,p90,p99,max_days
0,2,13,34,37


Global reach (how many countries a video trends in)

In [17]:
q("""
WITH per_video AS (
  SELECT video_id, count(DISTINCT video_trending_country) AS countries
  FROM trending
  GROUP BY 1
)
SELECT
  approx_quantile(countries, 0.50) AS p50,
  approx_quantile(countries, 0.90) AS p90,
  approx_quantile(countries, 0.99) AS p99,
  max(countries) AS max_countries
FROM per_video;
""")


,p50,p90,p99,max_countries
0,1,5,18,108


find the “most global” videos

In [18]:
q("""
SELECT
  video_id,
  max(video_title) AS video_title,
  max(channel_title) AS channel_title,
  count(DISTINCT video_trending_country) AS countries
FROM trending
GROUP BY 1
ORDER BY countries DESC
LIMIT 20;
""")


,video_id,video_title,channel_title,countries
0,XRgOPWisuog,EXTREME SHARK GAME!!!,Sylex,108
1,CgCVZdcKcqY,BLACKPINK - ‘뛰어(JUMP)’ M/V,BLACKPINK,108
2,zs7ISrSdVUI,Ellie Goulding - Love Me Like You Do (Lyrics Cover) | Pop Mage,Pop Mage,106
3,k2hXVxvZP7E,REAL MADRID 2 - 1 FC BARCELONA | RESUMEN LALIGA EA SPORTS,LALIGA EA SPORTS,106
4,zPbrkmdcTfo,1000 Players Simulate Civilization: Boys vs Girls,MrBeast Gaming,105
5,zEzyD5oszwM,skibidi toilet 79 (part 2),DaFuq!?Boom!,105
6,DWCl2dN6hpg,1 Pro vs 500 Hunters,MrBeast Gaming,105
7,npNmyb-qqGw,"Survive 1000 Days, Win $100,000",MrBeast Gaming,104
8,NED7nev2ywQ,"Stray Kids ""Do It"" M/V",JYP Entertainment,103
9,qTMKHZelGAs,We Built New York In Minecraft,MrBeast Gaming,102


stickiest in a country

In [20]:
q("""
WITH pvc AS (
  SELECT
    video_id,
    video_trending_country,
    count(DISTINCT video_trending_date) AS days_trended
  FROM trending
  GROUP BY 1,2
)
SELECT
  pvc.video_trending_country,
  pvc.video_id,
  ANY_VALUE(t.video_title) AS video_title,
  ANY_VALUE(t.channel_title) AS channel_title,
  pvc.days_trended
FROM pvc
JOIN trending t
  ON t.video_id = pvc.video_id
 AND t.video_trending_country = pvc.video_trending_country
GROUP BY
  pvc.video_trending_country,
  pvc.video_id,
  pvc.days_trended
ORDER BY pvc.days_trended DESC
LIMIT 20;
""")



FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,video_trending_country,video_id,video_title,channel_title,days_trended
0,Romania,b1cXgCFOiaE,Tzanca Uraganu ❌ Costel Biju - Meneaito [video oficial] 2025,Tzanca Uraganu,37
1,Hong Kong,Ox29z5Nf1Uk,"[#2024MAMA] G-DRAGON - 무제(Untitled, 2014)+POWER+HOME SWEET HOME+뱅뱅뱅+FANTASTI...",Mnet K-POP,37
2,Egypt,Nern2ReRazc,تحدي احزر التوأم الحقيقي🤔مع التوينز,TRIPLEZ - تريبلز,37
3,Saudi Arabia,rA3-rEdQ8Wo,24 ساعه بدون جوال 📵( أبها البهيه 😍),RAED,37
4,Bolivia,9fQ0tP6aOks,¡LE VOY A DAR UNA SORPRESA A MI FAMILIA!,SKabeche,37
5,Dominican Republic,vWRmfW9VMAU,Modificamos el SUPRA de Rápidos y Furiosos en 24 HORAS | Kenyi Nakamura,Kenyi Nakamura,37
6,Peru,P4YLOdr6e7U,BLESSD X NACIONAL OFICIAL | HIMNO DE LA GRANDEZA 💚🤍 (VIDEO OFICIAL),BLESSD EL BENDITO 💙,37
7,Iceland,x-I8ZHfwV9Q,"I Spent $10,000 Buying Expensive Games",LazarBeam,37
8,Dominican Republic,cY8B1wWtRm8,DONATY x BRAULIO FOGON - TAMBOREO (VIDEO OFICIAL),Donaty,37
9,Panama,ueaIQkxrAmY,ROBANDO COCHES del CHAVO del 8 en GTA 5! 🚘🧢 (Mods),E-MasterSensei,37
